In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import cv2

from generator import Generator
from BaseColor import BaseColor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
G = Generator()
G = G.to(device)
G.eval()  

def broj_parametara(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Broj parametara Generatora:", broj_parametara(G))


Broj parametara Generatora: 25155563


In [5]:
L_tensor = torch.rand(1, 1, 256, 256).to(device) * 2 - 1

with torch.no_grad():
    ab_output = G(L_tensor)

print("Oblik izlaza iz generatora (AB kanali):", ab_output.shape)
print("Min vrednost izlaza:", ab_output.min().item())
print("Max vrednost izlaza:", ab_output.max().item())


Oblik izlaza iz generatora (AB kanali): torch.Size([1, 2, 256, 256])
Min vrednost izlaza: -0.3720718026161194
Max vrednost izlaza: -0.26477399468421936
